In [1]:
import pandas as pd
import numpy as np
import torch

In [3]:
# 有向图edge关系，第0行follow第1行
# follower, friend, mention, reply, quote, URL, hashtag
edge_type = torch.load("edge_type.pt")  # (1700108)
edge_index = torch.load("edge_index.pt")

edge_index = torch.tensor(edge_index, dtype=torch.int64)  # (2, 1700108)
edge_index_0, edge_index_1 = edge_index[:, (0 == edge_type)], edge_index[:, (1 == edge_type)]
edge_index = np.concatenate([edge_index_0.flip(0), edge_index_1], axis=1)

C:\Users\TINKY2~1\AppData\Local\Temp/ipykernel_7476/2480114899.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  edge_index = torch.tensor(edge_index, dtype=torch.int64)  # (2, 1700108)


In [12]:
# neg (against)
stance = 0
bot_label = torch.load('labels_bot.pt') # (10199) 0-human, 1-bot
# neutral: 3776, against: 3637, support: 2786
sl = torch.load("labels_stance.pt")  # (10199) 0-neutral, 1-against, 2-support
outcome = sl.masked_fill(sl==0, 1).masked_fill(sl==1, 0) # -> 0-against, 2-support

N = len(outcome) # num nodes
treat_indicator, prop_label = np.zeros(N), np.zeros(N)
followings_dict = {}
for i in range(N):
    u, v = edge_index[0][i], edge_index[1][i]
    followings_dict.setdefault(u, []).append(v)
id_all = set(np.array([i for i in range(N)]))
id_s = id_all - set(np.nonzero(outcome-stance)[0])
prop_label[list(map(int, id_s))] = 1


print("constructing treat/control:...")
treat_id = []
control_id = []
for i in range(N):
    if i not in followings_dict.keys():
        continue
    if bot_label[i] == 1:
        continue
    for j in followings_dict[i]:
        if outcome[j] == stance:
            if bot_label[j] == 0:
                treat_id.append(i)
            elif bot_label[j] == 1:
                control_id.append(i)

treat_id, control_id = set(treat_id), set(control_id)
intersec = set(treat_id)&set(control_id)
treat_id = list(map(int, treat_id-intersec))
control_id = list(map(int, control_id-intersec))
print("len of treat/control/intersect:", len(treat_id), len(control_id), len(intersec))
treat_indicator[treat_id] = -1
treat_indicator[control_id] = 1

bot_label = bot_label.numpy()
y_truth = outcome.numpy()
print("b, t, prop, y shape: ", bot_label.shape, treat_indicator.shape, y_truth.shape, prop_label.shape)


constructing treat/control:...
len of treat/control/intersect: 1895 4 92
b, t, prop, y shape:  (10199,) (10199,) (10199,) (10199,)


In [64]:
# np.save('twi22/t1_covid/t1_edge.npy', edge_index)
# np.save('twi22/t1_covid/t1_bot_label.npy', bot_label)
# np.save('twi22/t1_covid/t1_T_label.npy', treat_label)
# np.save('twi22/t1_covid/t1_y.npy', y_truth)
# np.save('twi22/t1_covid/t1_prop_label.npy', prop_label)

# np.save('twi22/t2_war/t2_edge.npy', edge_index)
# np.save('twi22/t2_war/t2_bot_label.npy', bot_label)
# np.save('twi22/t2_war/t2_T_label.npy', treat_label)
# np.save('twi22/t2_war/t2_y.npy', y_truth)
# np.save('twi22/t2_war/t2_prop_label.npy', prop_label)

np.save('twi22/t3_climate/t3_edge.npy', edge_index)
np.save('twi22/t3_climate/t3_bot_label.npy', bot_label)
np.save('twi22/t3_climate/t3_T_label.npy', treat_label)
np.save('twi22/t3_climate/t3_y.npy', y_truth)
np.save('twi22/t3_climate/t3_prop_label.npy', prop_label)